# 收集数据

In [1]:
import pandas as pd
from numpy import *
import tensorflow as tf

G:\study\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
ratings_df = pd.read_csv(r'D:\dataSet\movielens\ratings.csv')
movies_df = pd.read_csv(r'D:\dataSet\movielens\movies.csv')
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [4]:
movies_df['movieRow'] = movies_df.index
movies_df.head()

,movieId,title,genres,movieRow
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,1
2,3,Grumpier Old Men (1995),Comedy|Romance,2
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,3
4,5,Father of the Bride Part II (1995),Comedy,4


## 筛选movies_df中的特征

In [5]:
movies_df = movies_df[['movieRow', 'movieId', 'title']]
movies_df.head()

,movieRow,movieId,title
0,0,1,Toy Story (1995)
1,1,2,Jumanji (1995)
2,2,3,Grumpier Old Men (1995)
3,3,4,Waiting to Exhale (1995)
4,4,5,Father of the Bride Part II (1995)


In [6]:
ratings_df = pd.merge(ratings_df, movies_df, on = 'movieId')
ratings_df.head()

,userId,movieId,rating,timestamp,movieRow,title
0,1,31,2.5,1260759144,30,Dangerous Minds (1995)
1,7,31,3.0,851868750,30,Dangerous Minds (1995)
2,31,31,4.0,1273541953,30,Dangerous Minds (1995)
3,32,31,4.0,834828440,30,Dangerous Minds (1995)
4,36,31,3.0,847057202,30,Dangerous Minds (1995)


In [7]:
ratings_df = ratings_df[['userId', 'movieRow', 'rating']]
ratings_df.head()

,userId,movieRow,rating
0,1,30,2.5
1,7,30,3.0
2,31,30,4.0
3,32,30,4.0
4,36,30,3.0


## 创建电影评分矩阵 rating 和评分记录矩阵 record

In [8]:
userNo = ratings_df['userId'].max() + 1
movieNo = ratings_df['movieRow'].max() + 1

In [9]:
userNo

672

In [10]:
movieNo

9123

In [11]:
rating = zeros((movieNo, userNo))   #每一行记录保存电影被所有用户的评分

In [12]:
flag = 0
ratings_df_length = shape(ratings_df)[0]

for index, row in ratings_df.iterrows():
    rating[int(row['movieRow']), int(row['userId'])] = row['rating']
    flag += 1
    print('processed %d, %d left' % (flag, ratings_df_length - flag))
    if flag == 10:
        break

processed 1, 100003 left
processed 2, 100002 left
processed 3, 100001 left
processed 4, 100000 left
processed 5, 99999 left
processed 6, 99998 left
processed 7, 99997 left
processed 8, 99996 left
processed 9, 99995 left
processed 10, 99994 left


In [13]:
record = rating > 0
record

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [14]:
record = array(record, dtype=int)
record

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [15]:
rating

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

# 构建模型

In [16]:
def normalizeRatings(rating, record):
    m, n = rating.shape   #m是用户数,n是电影数
    rating_mean = zeros((m, 1))
    rating_norm = zeros((m, n))
    for i in range(m):
        idx = record[i, :] != 0  #被用户评过分的电影
        rating_mean[i] = mean(rating[i, idx])   #每个用户对TA所看过的电影的评分的平均值
        rating_norm[i, idx] -= rating_mean[i]   #每部电影的评分减去该用户对所有电影的评分的平均值
    
    return rating_norm, rating_mean

In [17]:
rating_norm, rating_mean = normalizeRatings(rating, record)
rating_norm

G:\study\Anaconda\lib\site-packages\numpy\core\fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
G:\study\Anaconda\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

 有些行全为0,计算平均值时结果会是NaN

 将NaN转化为0

In [18]:
rating_norm = nan_to_num(rating_norm)
rating_norm

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [19]:
rating_mean = nan_to_num(rating_mean)
rating_mean

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

In [20]:
num_features = 10   #电影类型数
X_parameters = tf.Variable(tf.random_normal([movieNo, num_features], stddev = 0.35))
Theta_parameters = tf.Variable(tf.random_normal([userNo, num_features], stddev = 0.35))
loss  = 1/2 * tf.reduce_sum(((tf.matmul(X_parameters, Theta_parameters, transpose_b=True)- rating_norm) * record)**2) \
+  5/2 * (tf.reduce_sum(X_parameters**2) +  tf.reduce_sum(Theta_parameters**2))

#当lambda为1时,error是4000左右,当lambda为5时,error为122!,可见正则化项对性能的影响!
optimizer = tf.train.AdamOptimizer(1e-4)
train = optimizer.minimize(loss)

# 训练模型

In [21]:
tf.summary.scalar('loss', loss)

<tf.Tensor 'loss:0' shape=() dtype=string>

In [22]:
summaryMerged = tf.summary.merge_all()

In [23]:
filename = './movie_tensorboard'

In [24]:
writer = tf.summary.FileWriter(filename)

In [25]:
sess = tf.Session()

In [26]:
init = tf.global_variables_initializer()

In [27]:
sess.run(init)

In [ ]:
for i in range(5000):
    _, movie_summary = sess.run([train, summaryMerged])
    writer.add_summary(movie_summary, i)

# 评估模型

In [ ]:
Current_X_parameters, Current_Theta_parameters = sess.run([X_parameters, Theta_parameters])
predicts = dot(Current_X_parameters, Current_Theta_parameters.T) + rating_mean
#predicts的每一列是一个用户对所有电影的评分
errors = sqrt(sum((predicts-rating) **2))

In [ ]:
errors

# 构建完整的电影推荐系统

In [ ]:
user_id = input('')
sortedResult = predicts[:, int(user_id)].argsort()[::-1]

idx = 0
for i in sortedResult:
    print('评分:%.2f, 电影名: %s' %(predicts[i, int(user_id)], movies_df.iloc[i]['title']))
    idx += 1
    if idx == 20:   #推荐top20部电影
        break